<a href="https://colab.research.google.com/github/quanvu0996/data_science/blob/main/tf/gradient_tape1_vi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient descent with tensorflow
Các công cụ hỗ trợ xây dựng mô hình học máy, học sâu như tensorflow, pytorch ngày càng phổ biến. Xây dựng mô hình trở nên dễ dàng chỉ với một câu lệnh fit. Tuy nhiên để tối ưu hoặc sáng tạo kiến trúc mới người ta cần một công cụ linh động hơn như Gradient tape để huấn luyện các model có kiến trúc đặc biệt.

In [3]:
import tensorflow as tf
tf.__version__

'2.8.2'

### Một số khái niệm cơ bản của tensorflow
**Hằng**: là một tensor có giá trị cố định. Có thể là một số (scalar- tensor 1 chiều), hoặc ma trận (tensor 2 chiều), hoặc tensor n-chiều

In [4]:
a = tf.constant(1)
b = tf.constant([3, 5])
c = tf.ones(shape=(2,2))

print(a, b, c)

tf.Tensor(1, shape=(), dtype=int32) tf.Tensor([3 5], shape=(2,), dtype=int32) tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


**Biến**: là một đối tượng có thể thay đổi giá trị tùy thuộc dữ liệu đầu vào.

In [5]:
X = tf.Variable(initial_value= tf.zeros((2, 2)), trainable= True)
X

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[0., 0.],
       [0., 0.]], dtype=float32)>

Những khái nhiệm này cũng tương ứng với các khái niệm trong đại số tuyến tính. <br>
**Gradient tape** là công cụ tính đạo hàm của tensorflow. Cú pháp sau tính toán đạo hàm của hàm $y=x^2$ tại $x=4$, $(\frac{δy}{δx| x=4} = 8)$:

In [6]:
x = tf.constant(4.0)

with tf.GradientTape() as tape:
    tape.watch(x) # x không phải là một tf.Variable => cần chỉ biến cần tape theo dõi, nếu không giá trị đạo hàm sẽ là 0.
                  # nếu x là tf.Variable thì không cần watch
    y = x ** 2
dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(8.0, shape=(), dtype=float32)


Thực hiện đạo hàm cấp 2 với cú pháp như sau:

In [7]:
x = tf.constant(4.0)

with tf.GradientTape() as t2:
    t2.watch(x)
    with tf.GradientTape() as t:
        t.watch(x)
        y = x ** 2
    dy_dx = t.gradient(y, x)
d2y_dx = t2.gradient(dy_dx, x)
print(d2y_dx)

tf.Tensor(2.0, shape=(), dtype=float32)


## Simple gradient descent pipeline

Ta sẽ xem xét một ví dụ xây dựng một mô hình hồi quy tuyến tính đơn giản:
$Y = aX + b$ <br>



In [174]:
X = tf.constant([1., 4., 6, 3, 3, 4, 5, 6, 7])
Y = tf.constant([0.25, 1.2, 0.79, 0.52, 1.6, 1.7, 1.9, 2, 2])

Trong ví dụ này, $a$ và $b$ là các tham số của mô hình, chúng ta muốn tìm kiếm một giá trị tối ưu của chúng để mô hình khái quát hóa được mối quan hệ của 2 biến $X$, $Y$. Vì vậy, $a$ và $b$ trong ví dụ này là 2 biến một chiều (shape = (1,)). Ta thực hiện khai báo:

In [175]:
a = tf.Variable(initial_value=0., trainable= True)
b = tf.Variable(initial_value= 0., trainable= True)

X và Y trong trường hợp này đại diện cho phần dữ liệu huấn luyện, vì vậy, chúng là các hằng số đã biết. Kích thước của chúng phụ thuộc chiều dài của batch size. <br>
Ta định nghĩa một hàm loss đơn giản:

In [176]:
def loss_fn(y_true, y_pred):
    return tf.reduce_sum(tf.square(y_true-y_pred))

Để thực hiện huấn luyện mô hình, ta thực hiện các bước: tính giá trị dự đoán, tính giá trị hàm loss, tính đạo hàm riêng của hàm loss theo từng tham số, cập nhật lại giá trị tham số. Triển khai với Gradient tape có dạng như sau:

In [177]:
learning_rate = 0.001

with tf.GradientTape(persistent =True) as tape:
    y_pred = a*X+b
    loss = loss_fn(Y, y_pred)

# Tính đạo hàm riêng của loss theo từng tham số
a_gradient = tape.gradient(loss, a)
b_gradient = tape.gradient(loss, b)

# Cập nhật lại giá trị các tham số: w1 = w0 - learning_rate * d(loss)/dw
a.assign_sub(a_gradient*learning_rate)
b.assign_sub(b_gradient*learning_rate)

print(a)
print(b)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.116900004>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.023920001>


Giá trị persistent =True cho phép tính được giá trị đạo hàm (tape.gradient) cho nhiều hơn 1 lần. <br>
Để quan sát quá trình tối ưu, ta huấn luyện với nhiều epoch và kiểm tra giá trị hàm loss:

In [178]:
def train_step(x_true, y_true):
    with tf.GradientTape(persistent =True) as tape:
        # Tính giá trị dự đoán và giá trị hàm loss
        y_pred = a*X+b
        loss = loss_fn(Y, y_pred)
        print("Loss: ", loss.numpy())

    # Tính đạo hàm riêng của loss theo từng tham số
    a_gradient = tape.gradient(loss, a)
    b_gradient = tape.gradient(loss, b)

    # Cập nhật lại giá trị các tham số: w1 = w0 - learning_rate * d(loss)/dw
    a.assign_sub(a_gradient*learning_rate)
    b.assign_sub(b_gradient*learning_rate)
    print("model: Y= %s X + %s"%(a.numpy(), b.numpy()))

for i in range(5):
    print("Epoch: ", i)
    train_step(X, Y)

Epoch:  0
Loss:  8.134604
model: Y= 0.18587564 X + 0.038291242
Epoch:  1
Loss:  4.186867
model: Y= 0.22655392 X + 0.0470237
Epoch:  2
Loss:  2.8102624
model: Y= 0.25052384 X + 0.052426066
Epoch:  3
Loss:  2.330071
model: Y= 0.2646282 X + 0.055861536
Epoch:  4
Loss:  2.1624107
model: Y= 0.2729075 X + 0.05813503


Có thể thấy giá trị hàm loss giảm theo thời gian và giá trị a, b dần hội tụ trở nên ổn định qua các epochs.<br>
Hàm trên sử dụng Gradient descent cơ bản, trong thực tế ta sẽ mong muốn sử dụng các optimizer hiệu quả hơn. Ngoài ra, số lượng tham số của một mạng học sâu trong thực tế có thể lên đến hàng triệu tham số, và ta không muốn phải cập nhật tay cho từng tham số một. Khi đó, ta sẽ làm như sau:

In [235]:
model = tf.keras.Sequential([
                           tf.keras.layers.Dense(1, kernel_initializer='zeros', bias_initializer='zeros')
])

optimizer= tf.optimizers.Adam(learning_rate = .07)

def train_step2(x_true, y_true):
    with tf.GradientTape(persistent =True) as tape:
        # Tính giá trị dự đoán và giá trị hàm loss
        y_pred = model(tf.expand_dims(X,-1))
        loss = loss_fn( tf.expand_dims(Y, -1), y_pred)
        print("Loss: ", loss.numpy())

    # Tính đạo hàm riêng của loss theo từng tham số
    variables = model.trainable_variables 
    gradients = tape.gradient(loss, variables)

    # Cập nhật lại giá trị các tham số:
    optimizer.apply_gradients(zip(gradients, variables))

    
for i in range(5):
    print("Epoch: ", i)
    train_step2(X, Y)

Epoch:  0
Loss:  19.457
Epoch:  1
Loss:  10.9911995
Epoch:  2
Loss:  5.402389
Epoch:  3
Loss:  2.5634406
Epoch:  4
Loss:  2.0542033
